In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
# !pip install transformers beautifulsoup4 requests

In [3]:
# !pip install mitosheet

In [7]:
# import mitosheet

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
import requests
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'transformers'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

E:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP PC\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [ ]:
tokens

tensor([[  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
         11838,   102]])

In [ ]:
result = model(tokens)


In [ ]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
tokens1=tokenizer.encode("I hate this, worst food got", return_tensors='pt')

In [ ]:
result2=model(tokens1)
result2

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.7017,  1.6676, -0.7510, -2.7241, -2.0983]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
result2.logits

tensor([[ 4.7017,  1.6676, -0.7510, -2.7241, -2.0983]],
       grad_fn=<AddmmBackward0>)

In [ ]:
result.logits


tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [ ]:
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [ ]:
int(torch.argmax(result.logits)+1)

4

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
regex = re.compile('.*comment.*')
regex

re.compile(r'.*comment.*', re.UNICODE)

In [ ]:
results = soup.find_all('p', {'class':regex})

reviews = [result.text for result in results]
reviews[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,We came for brunch and they ran out of seven s...
3,The food was delicious. The ricotta pancakes w...
4,Great place with delicious food and friendly s...
5,Great food amazing coffee and tea. Short walk ...
6,"Great service, lovely location, and really ama..."
7,Ricotta hot cakes! These were so yummy. I ate ...
8,We came for brunch twice in our week-long visi...
9,It was ok. Had coffee with my friends. I'm new...


In [ ]:
df.iloc[1]

review    Six of us met here for breakfast before our wa...
Name: 1, dtype: object

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[3])

4

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,We came for brunch and they ran out of seven s...,2
3,The food was delicious. The ricotta pancakes w...,4
4,Great place with delicious food and friendly s...,5
5,Great food amazing coffee and tea. Short walk ...,5
6,"Great service, lovely location, and really ama...",5
7,Ricotta hot cakes! These were so yummy. I ate ...,5
8,We came for brunch twice in our week-long visi...,4
9,It was ok. Had coffee with my friends. I'm new...,3
